
<img src="https://media.theresanaiforthat.com/icons/mistral-ai.svg?height=207" width="30" height="30">
<br>
<h1 align = 'center'>Test de d'appel de fonction python</h1>

---

Ce notebook vise à me familiariser avec un exemple simple d'appel de fonction Python dans le cadre d'une discussion textuelle avec un modèle `Mistral`.
<br><br>
L'appel de fonctions est une fonctionnalité qui permet aux modèles Mistral de se connecter à des outils externes. En intégrant ces modèles à des outils externes tels que des fonctions ou des API définies par l'utilisateur, nous pouvons facilement créer des applications qui répondent à des cas d'utilisation spécifiques et résolvent des problèmes pratiques. Par exemple, dans ce guide, nous avons développé plusieurs fonctions pour naviguer dans une base de donnée. cela permet d'avoir des interaction plus fine et transparente avec la base de données.
<br><br>
**_Cette version du notebook vous permet de tester l'appel de fonction avec des API externes, y compris une API nécessitant une authentification._**
<br><br>
il y a quatre étapes avec l'appel de fonction :

- **Utilisateur** : spécifier les outils et la requête

- **Modèle** : Générer des arguments de fonction le cas échéant

- **Utilisateur** : Exécuter la fonction pour obtenir les résultats de l'outil

- **Modèle** : Générer la réponse finale

## Chargement des packages :

In [104]:
import pandas as pd
import functools
import requests
import json
from mistralai.client import MistralClient
from requests.auth import HTTPBasicAuth
from mistralai.models.chat_completion import ChatMessage

# Création de la base de données :

In [105]:
# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV file
df.to_csv('items.csv', index=False)

Cette base de données sera utilisée par le modèle pour répondre aux questions. Pour ce faire, il est nécessaire de créer une série de fonctions que Mistral utilisera pour interroger la base de données. 
<br><br>
Ces fonctions doivent être conçues de manière à ce qu'elles puissent extraire un fichier JSON exploitable par le modèle.

# Déclaration des fonctions :

In [106]:
def retrieve_payment_status(transaction_id: str) -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/" + transaction_id).json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps({'status': data['Payment_status']}, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(transaction_id: str) -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/" + transaction_id).json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps({'date': data['Payment_status']}, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_max() -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/max").json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps(data, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_min() -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/min").json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps(data, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_count(password : str) -> str: #password : str
    data = requests.api.get("http://127.0.0.1:8000/Transaction/count", auth=HTTPBasicAuth("your_username", password)).json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps({'count' : data}, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

Pour que les modèles Mistral puissent comprendre et utiliser les fonctions, il est nécessaire de décrire les spécifications de chaque fonction à l'aide d'un schéma JSON. Cela implique de définir le type, le nom, la description, les paramètres et les paramètres requis pour chaque fonction. Dans notre cas, nous avons deux fonctions à spécifier, nous devons donc énumérer deux spécifications de fonction dans une liste.

Actuellement, je ne connais pas de méthode permettant de réaliser cette étape automatiquement et rapidement.

In [107]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_max",
            "description": "Get payment max of all transactions",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_min",
            "description": "Get payment min of all transactions",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_count",
            "description": "Get payment count of all transactions",
            "parameters": {
                "type": "object",
                "properties": {
                    "password": {
                        "type": "string",
                        "description": "The password.",
                    }
                },
                "required": ['password'],
            },
        },
    },
]

In [108]:
username = "your_username"
password = "your_password"
data = requests.api.get("http://127.0.0.1:8000/Transaction/count", auth=HTTPBasicAuth(username, password)).json()
data

5

Ensuite, nous devons organiser les deux fonctions dans un dictionnaire, où les clés représentent le nom de chaque fonction et les valeurs correspondent aux fonctions elles-mêmes, avec le dataframe défini en tant que paramètre. Cette organisation nous permet d'appeler chaque fonction en utilisant son nom de fonction.

Cette étape permettra au modèle de sélectionner et d'appeler la bonne fonction en fonction de la question posée.

In [109]:
names_to_functions = {
    'retrieve_payment_status': functools.partial(retrieve_payment_status),
    'retrieve_payment_date': functools.partial(retrieve_payment_date),
    'retrieve_payment_max': functools.partial(retrieve_payment_max),
    'retrieve_payment_min': functools.partial(retrieve_payment_min),
    'retrieve_payment_count': functools.partial(retrieve_payment_count),
}

Nous allons maintenant initialiser la conversation avec le modèle Mistral.

In [110]:
messages = [
    ChatMessage(role="system", content="ta langue de discussion est le français"),
    ChatMessage(role="user", content="combien y'a il de transcation actuellement?")
]

# Selection de la fonction a appeler
Initialisation du client et appel de réponse client

In [111]:
model = "mistral-large-latest"
api_key="dXhF7z625frqL76yHgWY5PZJ9l0298mg" # ne fonctionne plus après le 31 mars 2024

client = MistralClient(api_key=api_key)
response = client.chat(model=model, messages=messages, tools=tools, tool_choice="auto")
print(response.choices[0].message.content)

To get the count of all transactions, I need to call the `retrieve_payment_count` function. However, it requires a password. Could you please provide the password?


In [112]:
messages.append(response.choices[0].message)
messages.append(ChatMessage(role="user", content="Mon mot de passe est 'your_password'"))
response = client.chat(model=model, messages=messages, tools=tools, tool_choice="auto")
messages.append(response.choices[0].message)

# Execution de la fonction selectionnée

In [113]:
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_count 
function_params:  {'password': 'your_password'}


In [114]:
function_result = names_to_functions[function_name](**function_params)
function_result

'{\n  "count": 5\n}'

In [115]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result))

response = client.chat(model=model, messages=messages)
messages.append(response.choices[0].message)
print(response.choices[0].message.content)

Il y a actuellement 5 transactions.


**Conclusion** : Ce notebook démontre la faisabilité d'un outil d'exploration de données basé sur un assistant textuel. Actuellement, l'exécution du code est manuelle et locale, mais il est possible de modifier le code pour automatiser cette étape et offrir une expérience transparente avec le modèle. De plus, il est possible d'améliorer les interactions entre la base de données et le modèle en ajoutant des fonctions supplémentaires.